<a href="https://colab.research.google.com/github/shuen1106lu/Program-Language/blob/main/%E7%AC%AC%E5%85%AB%E5%91%A8_%E6%96%87%E5%AD%97%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**關於「幻想鄉」的描述與簡介，及常見字詞的特色**

In [1]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [2]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1D072ZNIrEiR3FKym76eQ_R0ywGVsGh6EddzmnMnVV5M/edit?usp=sharing').sheet1

In [10]:
dicts = gc.open_by_url('https://docs.google.com/spreadsheets/d/1D072ZNIrEiR3FKym76eQ_R0ywGVsGh6EddzmnMnVV5M/edit?usp=sharing').get_worksheet(1)
dicts = dicts.get_all_records()
dicts = pd.DataFrame(dicts)

WorksheetNotFound: index 1 not found

In [4]:
# 讀取所有數據
rows = gsheets.get_all_records()
import pandas as pd
df = pd.DataFrame(rows)

In [5]:
df.head()

項目                                                 回答
0       前言  「幻想鄉」是日本同人遊戲系列《東方Project》（由ZUN／太田順也創作）的核心舞台，是一...
1  地理位置與結界  幻想鄉原本位於日本本州東北部的深山（現實中的岐阜縣白川鄉附近），但隨著外界科學文明的發展，1...
2     社會結構  人類與非人類共存：人類村落（主要位於「人間之里」）與妖怪、神明、妖精等超自然生物形成微妙的平...
3     人間之里  普通人類居住地，受慧音（半獸白澤）的歷史知識保護，避免被妖怪侵擾。村民對妖怪既恐懼又依賴（如...
4     妖怪之山  天狗（新聞社《文々。新聞》）、河童（科技研發）、守矢神社（外來神明風神諏訪子與八坂神奈子）的...

In [6]:
df.describe()

項目                                                 回答
count   30                                                 30
unique  30                                                 30
top     前言  「幻想鄉」是日本同人遊戲系列《東方Project》（由ZUN／太田順也創作）的核心舞台，是一...
freq     1                                                  1

In [7]:
!pip install jieba
!pip install snownlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 31.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760946 sha256=635db5ec9c59916cc9caefca992cfa180492320e87fdf786de79007d6ab34b43
  Stored in directory: /root/.cache/pip/wheels/4a/fc/04/d1937c02b2a445b34754da55f71612a3df648a38d711bd17eb
Successfully built snownlp


In [8]:
import jieba
from snownlp import SnowNLP
import pandas as pd
from collections import Counter
import re
import plotly.express as px

In [9]:
for _, row in dicts.iterrows():
    word = row['words']
    jieba.add_word(word, freq=10000)
    jieba.suggest_freq(word, tune=True)

NameError: name 'dicts' is not defined

In [11]:
# 只保留中文字
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))

In [12]:
df['Chinese_Content'] = df['Cleaned_Content'].apply(extract_chinese)

# 使用 Jieba 斷詞
df['Tokenized'] = df['Chinese_Content'].apply(lambda x: list(jieba.cut(x, HMM=True)))

# 展平成所有詞語的列表
all_words = [word for tokens in df['Tokenized'] for word in tokens if len(word) > 1]

KeyError: 'Cleaned_Content'

In [13]:
df['Chinese_Content']

KeyError: 'Chinese_Content'

In [14]:
df['Tokenized']

KeyError: 'Tokenized'

In [15]:
word_counts = Counter(all_words)
word_counts

NameError: name 'all_words' is not defined

In [17]:
# 排除停用詞（這裡提供一個簡單的停用詞列表，可根據需要擴展）
stop_words = set(['所以', '好', '因為', '大家', '的', '是', '了', '我', '也', '在', '和', '就', '不', '有', '他', '她', '你', '我們', '這個'])
filtered_word_counts = {word: count for word, count in word_counts.items() if word not in stop_words}

NameError: name 'word_counts' is not defined

In [16]:
# 將詞頻轉為 DataFrame 並排序
word_freq_df = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])
word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

# 繪製關鍵字統計的條形圖
fig = px.bar(word_freq_df.head(30), x='Word', y='Frequency',
             title='關鍵字出現次數統計（前20名）',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

NameError: name 'filtered_word_counts' is not defined

In [ ]:
word_freq_df

Word  Frequency
0     高中        176
1     程式        157
2     希望        111
3     學習         62
4     可以         54
..   ...        ...
963  因為現          1
964   想試          1
965  謝承恩          1
966   藍若          1
967   感善          1

[968 rows x 2 columns]

In [ ]:
!apt-get update -qq
!apt-get install -y fonts-wqy-zenhei

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-wqy-zenhei
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 7,472 kB of archives.
After this operation, 16.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-wqy-zenhei all 0.9.45-8 [7,472 kB]
Fetched 7,472 kB in 1s (9,390 kB/s)
Selecting previously unselected package fonts-wqy-zenhei.
(Reading database ... 123630 files and directories currently installed.)
Preparing to unpack .../fonts-wqy-zenhei_0.9.45-8_all.deb ...
Unpacking fonts-wqy-zenhei (0.9.45-8) ...
Setting up fonts-wqy-zenhei (0.9.45-8) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [ ]:
from wordcloud import WordCloud

# 取前 300 個最常出現的詞
top_words = word_freq_df.head(300)

# 使用 WQY Zenhei 字體
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    font_path='/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc'
)

# 根據詞頻生成文字雲
wordcloud.generate_from_frequencies(dict(zip(top_words['Word'], top_words['Frequency'])))

# 將文字雲轉換為圖像
wordcloud_img = wordcloud.to_array()

# 使用 Plotly 顯示文字雲
fig = px.imshow(wordcloud_img)
fig.update_layout(
    title="文字雲",
    xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    coloraxis_showscale=False
)
fig.show()

In [ ]:
# 按日期計算討論熱度
df['Date'] = pd.to_datetime(df['Date'])
heatmap_data = df.groupby('Date').size().reset_index(name='Discussion_Count')

# 繪製趨勢圖
fig = px.line(heatmap_data, x='Date', y='Discussion_Count',
              title='不同時間段的討論熱度趨勢',
              labels={'Date': '日期', 'Discussion_Count': '討論數量'},
              markers=True)

fig.update_layout(
    xaxis_title="日期",
    yaxis_title="討論數量",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [ ]:
from google.colab import userdata
apikey = userdata.get('GOOGLE_API_KEY')

In [ ]:
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

prompt = f"請摘要出這些留言的重點，並補上重點中的情緒表現，把最多人呈現的情緒及其相關的內容整併在一起，說明這些發文的風向：{df.to_string()}"

response = model.generate_content(prompt)
print(response.text)

這些留言主要來自高中生，以及少數大學生、老師和非學生人士，他們都參與了同一門Python與AI入門課程。留言內容包含自我介紹、參與課程原因和Colab程式連結。

**最多人呈現的情緒：**熱情和期待 (積極正面)。

**相關內容：**

* **對AI和程式設計的濃厚興趣：**  大多數學生表達了對AI和程式設計的強烈興趣，許多人提到AI是當前趨勢，他們想學習程式設計技能以應用於未來生活、工作或升學（例如：想用AI寫作業、設計遊戲、提升工作效率、未來從事科技業等），展現出學習的熱情和對未來的期待。  部分學生雖然沒有程式設計基礎，但也抱持著積極嘗試的心態。

* **希望藉由課程提升技能：** 學生們普遍希望藉由這門課提升程式設計技能，以及對AI的理解，有些學生甚至希望能以此技能賺錢。這也反映了他們對未來發展的規劃和積極學習的態度。

* **老師和助教的鼓勵氛圍：** 助教們的留言營造了輕鬆友善的學習環境，鼓勵學生們互相交流，並舒緩學生們的學習壓力。

**風向總結：**

整體留言風向非常正面積極。學生們對課程內容充滿期待，展現了學習的熱情，也積極參與討論和互動。老師和助教的互動也為課程營造了良好的氛圍。  少數留言提到想利用程式設計賺錢，但整體而言，並非課程的主要目的或負面風向，而是學生們對未來發展的個人期許。

